In [1]:
import time, numpy as np

3a

In [9]:
Ps={
    **{chr(i+ord('a')):i+1 for i in range(26)},
    **{chr(i+ord('A')):i+27for i in range(26)}
}
with open('in.txt') as f:
    out=0
    for l in f.read().split('\n'):
        i=len(l)//2
        A,B=l[:i],l[i:]
        s=list(set(list(A))&set(list(B)))[0]
        #print(s)
        out+=Ps[s]
    print(out)

7795


3b

In [12]:
with open('in.txt') as f:
    ls=f.read().split('\n')
    out=0
    for li in range(0,len(ls),3):
        A,B,C=ls[li:li+3]
        s=list(set(list(A))&set(list(B))&set(list(C)))[0]
        #print(s)
        out+=Ps[s]
    print(out)

2703


2a

In [6]:
with open('in.txt') as f:
    scr=0
    for l in f.read().split('\n'):
        elf,me=l.split(' ')
        scr+=({'X':1,'Y':2,'Z':3}[me]
              +{
                  'X':{'A':3,'B':0,'C':6},
                  'Y':{'A':6,'B':3,'C':0},
                  'Z':{'A':0,'B':6,'C':3},
               }[me][elf])
    print(scr)

12740


2b

In [9]:
with open('in.txt') as f:
    scr=0
    for l in f.read().split('\n'):
        elf,ret=l.split(' ')
        me={
            'X':{'A':'Z','B':'X','C':'Y'},
            'Y':{'A':'X','B':'Y','C':'Z'},
            'Z':{'A':'Y','B':'Z','C':'X'},
        }[ret][elf]
        scr+=({'X':1,'Y':2,'Z':3}[me]
              +{
                  'X':{'A':3,'B':0,'C':6},
                  'Y':{'A':6,'B':3,'C':0},
                  'Z':{'A':0,'B':6,'C':3},
               }[me][elf])
    print(scr)

11980


1a

In [13]:
with open('in.txt') as f:
    groups=[list(map(int,g.split('\n')))
            for g in f.read().split('\n\n')]
    #print(groups)
    print(max(map(sum,groups)))

75501


1b

In [14]:
print(sum(sorted(map(sum,groups),reverse=True)[:3]))

215594
